# LR

In [75]:
from pyspark.ml.classification import LogisticRegression

from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import lit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

import numpy as np
import pandas as pd


In [2]:
train_data = spark.read.text('C:\spark\data\my_data\iris_data.txt')
train_data.head(5)
# type(train_data)


[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa')]

In [3]:
train_data.collect()

[Row(value='5.1,3.5,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.0,1.4,0.2,Iris-setosa'),
 Row(value='4.7,3.2,1.3,0.2,Iris-setosa'),
 Row(value='4.6,3.1,1.5,0.2,Iris-setosa'),
 Row(value='5.0,3.6,1.4,0.2,Iris-setosa'),
 Row(value='5.4,3.9,1.7,0.4,Iris-setosa'),
 Row(value='4.6,3.4,1.4,0.3,Iris-setosa'),
 Row(value='5.0,3.4,1.5,0.2,Iris-setosa'),
 Row(value='4.4,2.9,1.4,0.2,Iris-setosa'),
 Row(value='4.9,3.1,1.5,0.1,Iris-setosa'),
 Row(value='5.4,3.7,1.5,0.2,Iris-setosa'),
 Row(value='4.8,3.4,1.6,0.2,Iris-setosa'),
 Row(value='4.8,3.0,1.4,0.1,Iris-setosa'),
 Row(value='4.3,3.0,1.1,0.1,Iris-setosa'),
 Row(value='5.8,4.0,1.2,0.2,Iris-setosa'),
 Row(value='5.7,4.4,1.5,0.4,Iris-setosa'),
 Row(value='5.4,3.9,1.3,0.4,Iris-setosa'),
 Row(value='5.1,3.5,1.4,0.3,Iris-setosa'),
 Row(value='5.7,3.8,1.7,0.3,Iris-setosa'),
 Row(value='5.1,3.8,1.5,0.3,Iris-setosa'),
 Row(value='5.4,3.4,1.7,0.2,Iris-setosa'),
 Row(value='5.1,3.7,1.5,0.4,Iris-setosa'),
 Row(value='4.6,3.6,1.0,0.2,Iris-setosa'),
 Row(value=

In [4]:
train_data = spark.read.csv('C:\spark\data\my_data\iris_data.csv', header=False)
train_data.head(5)

[Row(_c0='5.1', _c1='3.5', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.9', _c1='3.0', _c2='1.4', _c3='0.2', _c4='1.0'),
 Row(_c0='4.7', _c1='3.2', _c2='1.3', _c3='0.2', _c4='1.0'),
 Row(_c0='4.6', _c1='3.1', _c2='1.5', _c3='0.2', _c4='1.0'),
 Row(_c0='5.0', _c1='3.6', _c2='1.4', _c3='0.2', _c4='1.0')]

In [5]:
train_data.show(5)

+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [6]:
train_data.columns

['_c0', '_c1', '_c2', '_c3', '_c4']

In [7]:
train_data.select('_c0').show(10)

+---+
|_c0|
+---+
|5.1|
|4.9|
|4.7|
|4.6|
|5.0|
|5.4|
|4.6|
|5.0|
|4.4|
|4.9|
+---+
only showing top 10 rows



In [36]:

train_data = train_data.withColumn('_c0', train_data['_c0'].cast('double'))\
       .withColumn('_c1', train_data['_c1'].cast('double'))\
       .withColumn('_c2', train_data['_c2'].cast('double'))\
       .withColumn('_c3', train_data['_c3'].cast('double'))
input_col = ['_c0', '_c1', '_c2', '_c3']
vecAssembler = VectorAssembler(inputCols=input_col, outputCol="features")
new_train_data = vecAssembler.transform(train_data)
stringIndexer = StringIndexer(inputCol="_c4", outputCol="label")
pipeline = Pipeline(stages=[vecAssembler, stringIndexer])
pipelineFit = pipeline.fit(train_data)
dataset = pipelineFit.transform(train_data)

In [37]:
new_train_data

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector]

In [38]:
new_train_data.show(5)

+---+---+---+---+---+-----------------+
|_c0|_c1|_c2|_c3|_c4|         features|
+---+---+---+---+---+-----------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|
+---+---+---+---+---+-----------------+
only showing top 5 rows



In [34]:
train_data.show(5)

+---+---+---+---+---+
|_c0|_c1|_c2|_c3|_c4|
+---+---+---+---+---+
|5.1|3.5|1.4|0.2|1.0|
|4.9|3.0|1.4|0.2|1.0|
|4.7|3.2|1.3|0.2|1.0|
|4.6|3.1|1.5|0.2|1.0|
|5.0|3.6|1.4|0.2|1.0|
+---+---+---+---+---+
only showing top 5 rows



In [33]:
tra_df = train_data.select(['_c0', '_c1'])
tra_df

DataFrame[_c0: double, _c1: double]

In [28]:
dataset.show(10)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  0.0|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  0.0|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  0.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  0.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  0.0|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|  0.0|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|  0.0|
+---+---+---+---+---+-----------------+-----+
only showing top 10 rows



In [29]:
dataset.select('_c0')

DataFrame[_c0: double]

In [30]:
dataset

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double]

In [27]:
c_1 = dataset['_c0']
c_1

Column<b'_c0'>

In [19]:
col_v1 = dataset._c0
col_v1

Column<b'_c0'>

In [20]:
col_v2 = dataset._c1
col_v2

Column<b'_c1'>

In [22]:
col_v12 = col_v1 * col_v2
col_v12

Column<b'(_c0 * _c1)'>

In [23]:
new_dataset = dataset.withColumn('col_v12', col_v12)

In [24]:
new_dataset.show(10)

+---+---+---+---+---+-----------------+-----+------------------+
|_c0|_c1|_c2|_c3|_c4|         features|label|           col_v12|
+---+---+---+---+---+-----------------+-----+------------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  0.0|17.849999999999998|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  0.0|14.700000000000001|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  0.0|15.040000000000001|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0|             14.26|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  0.0|              18.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  0.0|21.060000000000002|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0|15.639999999999999|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|  0.0|              17.0|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|             12.76|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|  0.0|15.190000000000001|
+---+---+---+---+---+-----------------+-----+------------------+
only showing top 10 rows



In [10]:
col1 = dataset.select('_c0')
col1_pd = col1.toPandas()
col1_ar = col1_pd.values
col1_ar

array([[5.1],
       [4.9],
       [4.7],
       [4.6],
       [5. ],
       [5.4],
       [4.6],
       [5. ],
       [4.4],
       [4.9],
       [5.4],
       [4.8],
       [4.8],
       [4.3],
       [5.8],
       [5.7],
       [5.4],
       [5.1],
       [5.7],
       [5.1],
       [5.4],
       [5.1],
       [4.6],
       [5.1],
       [4.8],
       [5. ],
       [5. ],
       [5.2],
       [5.2],
       [4.7],
       [4.8],
       [5.4],
       [5.2],
       [5.5],
       [4.9],
       [5. ],
       [5.5],
       [4.9],
       [4.4],
       [5.1],
       [5. ],
       [4.5],
       [4.4],
       [5. ],
       [5.1],
       [4.8],
       [5.1],
       [4.6],
       [5.3],
       [5. ],
       [7. ],
       [6.4],
       [6.9],
       [5.5],
       [6.5],
       [5.7],
       [6.3],
       [4.9],
       [6.6],
       [5.2],
       [5. ],
       [5.9],
       [6. ],
       [6.1],
       [5.6],
       [6.7],
       [5.6],
       [5.8],
       [6.2],
       [5.6],
       [5.9],
      

In [11]:
col2 = dataset.select('_c1')
col2_pd = col2.toPandas()
col2_ar = col2_pd.values
col2_ar

array([[3.5],
       [3. ],
       [3.2],
       [3.1],
       [3.6],
       [3.9],
       [3.4],
       [3.4],
       [2.9],
       [3.1],
       [3.7],
       [3.4],
       [3. ],
       [3. ],
       [4. ],
       [4.4],
       [3.9],
       [3.5],
       [3.8],
       [3.8],
       [3.4],
       [3.7],
       [3.6],
       [3.3],
       [3.4],
       [3. ],
       [3.4],
       [3.5],
       [3.4],
       [3.2],
       [3.1],
       [3.4],
       [4.1],
       [4.2],
       [3.1],
       [3.2],
       [3.5],
       [3.1],
       [3. ],
       [3.4],
       [3.5],
       [2.3],
       [3.2],
       [3.5],
       [3.8],
       [3. ],
       [3.8],
       [3.2],
       [3.7],
       [3.3],
       [3.2],
       [3.2],
       [3.1],
       [2.3],
       [2.8],
       [2.8],
       [3.3],
       [2.4],
       [2.9],
       [2.7],
       [2. ],
       [3. ],
       [2.2],
       [2.9],
       [2.9],
       [3.1],
       [3. ],
       [2.7],
       [2.2],
       [2.5],
       [3.2],
      

In [12]:
coll = col1_ar * col2_ar
coll

array([[17.85],
       [14.7 ],
       [15.04],
       [14.26],
       [18.  ],
       [21.06],
       [15.64],
       [17.  ],
       [12.76],
       [15.19],
       [19.98],
       [16.32],
       [14.4 ],
       [12.9 ],
       [23.2 ],
       [25.08],
       [21.06],
       [17.85],
       [21.66],
       [19.38],
       [18.36],
       [18.87],
       [16.56],
       [16.83],
       [16.32],
       [15.  ],
       [17.  ],
       [18.2 ],
       [17.68],
       [15.04],
       [14.88],
       [18.36],
       [21.32],
       [23.1 ],
       [15.19],
       [16.  ],
       [19.25],
       [15.19],
       [13.2 ],
       [17.34],
       [17.5 ],
       [10.35],
       [14.08],
       [17.5 ],
       [19.38],
       [14.4 ],
       [19.38],
       [14.72],
       [19.61],
       [16.5 ],
       [22.4 ],
       [20.48],
       [21.39],
       [12.65],
       [18.2 ],
       [15.96],
       [20.79],
       [11.76],
       [19.14],
       [14.04],
       [10.  ],
       [17.7 ],
       [

In [13]:
coll_df = pd.DataFrame(coll)

In [14]:
type(coll_df)

pandas.core.frame.DataFrame

In [15]:
coll_spark_df = spark.createDataFrame(coll_df)

In [16]:
coll_spark_df

DataFrame[0: double]

In [17]:
type(coll_spark_df)

pyspark.sql.dataframe.DataFrame

In [129]:
coll_cl = coll_spark_df.collect()

In [132]:
coll_cl

[Row(0=17.849999999999998),
 Row(0=14.700000000000001),
 Row(0=15.040000000000001),
 Row(0=14.26),
 Row(0=18.0),
 Row(0=21.060000000000002),
 Row(0=15.639999999999999),
 Row(0=17.0),
 Row(0=12.76),
 Row(0=15.190000000000001),
 Row(0=19.980000000000004),
 Row(0=16.32),
 Row(0=14.399999999999999),
 Row(0=12.899999999999999),
 Row(0=23.2),
 Row(0=25.080000000000002),
 Row(0=21.060000000000002),
 Row(0=17.849999999999998),
 Row(0=21.66),
 Row(0=19.38),
 Row(0=18.36),
 Row(0=18.87),
 Row(0=16.56),
 Row(0=16.83),
 Row(0=16.32),
 Row(0=15.0),
 Row(0=17.0),
 Row(0=18.2),
 Row(0=17.68),
 Row(0=15.040000000000001),
 Row(0=14.879999999999999),
 Row(0=18.36),
 Row(0=21.32),
 Row(0=23.1),
 Row(0=15.190000000000001),
 Row(0=16.0),
 Row(0=19.25),
 Row(0=15.190000000000001),
 Row(0=13.200000000000001),
 Row(0=17.34),
 Row(0=17.5),
 Row(0=10.35),
 Row(0=14.080000000000002),
 Row(0=17.5),
 Row(0=19.38),
 Row(0=14.399999999999999),
 Row(0=19.38),
 Row(0=14.719999999999999),
 Row(0=19.61),
 Row(0=16.5),
 

In [133]:
coll_list = [i[0] for i in coll_cl]

In [134]:
coll_list

[17.849999999999998,
 14.700000000000001,
 15.040000000000001,
 14.26,
 18.0,
 21.060000000000002,
 15.639999999999999,
 17.0,
 12.76,
 15.190000000000001,
 19.980000000000004,
 16.32,
 14.399999999999999,
 12.899999999999999,
 23.2,
 25.080000000000002,
 21.060000000000002,
 17.849999999999998,
 21.66,
 19.38,
 18.36,
 18.87,
 16.56,
 16.83,
 16.32,
 15.0,
 17.0,
 18.2,
 17.68,
 15.040000000000001,
 14.879999999999999,
 18.36,
 21.32,
 23.1,
 15.190000000000001,
 16.0,
 19.25,
 15.190000000000001,
 13.200000000000001,
 17.34,
 17.5,
 10.35,
 14.080000000000002,
 17.5,
 19.38,
 14.399999999999999,
 19.38,
 14.719999999999999,
 19.61,
 16.5,
 22.400000000000002,
 20.480000000000004,
 21.39,
 12.649999999999999,
 18.2,
 15.959999999999999,
 20.79,
 11.76,
 19.139999999999997,
 14.040000000000001,
 10.0,
 17.700000000000003,
 13.200000000000001,
 17.689999999999998,
 16.24,
 20.77,
 16.799999999999997,
 15.66,
 13.640000000000002,
 14.0,
 18.880000000000003,
 17.08,
 15.75,
 17.08,
 18.56

In [135]:
lit(coll_list)

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.functions.lit.
: java.lang.RuntimeException: Unsupported literal type class java.util.ArrayList [17.849999999999998, 14.700000000000001, 15.040000000000001, 14.26, 18.0, 21.060000000000002, 15.639999999999999, 17.0, 12.76, 15.190000000000001, 19.980000000000004, 16.32, 14.399999999999999, 12.899999999999999, 23.2, 25.080000000000002, 21.060000000000002, 17.849999999999998, 21.66, 19.38, 18.36, 18.87, 16.56, 16.83, 16.32, 15.0, 17.0, 18.2, 17.68, 15.040000000000001, 14.879999999999999, 18.36, 21.32, 23.1, 15.190000000000001, 16.0, 19.25, 15.190000000000001, 13.200000000000001, 17.34, 17.5, 10.35, 14.080000000000002, 17.5, 19.38, 14.399999999999999, 19.38, 14.719999999999999, 19.61, 16.5, 22.400000000000002, 20.480000000000004, 21.39, 12.649999999999999, 18.2, 15.959999999999999, 20.79, 11.76, 19.139999999999997, 14.040000000000001, 10.0, 17.700000000000003, 13.200000000000001, 17.689999999999998, 16.24, 20.77, 16.7999

In [136]:
lit(0)

Column<b'0'>

In [127]:
coll_spark_col = lit(coll_spark_df)

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [69]:
dataset.columns

['_c0', '_c1', '_c2', '_c3', '_c4', 'features', 'label']

In [71]:
dataset.select(['_c0', '_c1']).show(5)

+---+---+
|_c0|_c1|
+---+---+
|5.1|3.5|
|4.9|3.0|
|4.7|3.2|
|4.6|3.1|
|5.0|3.6|
+---+---+
only showing top 5 rows



In [73]:
dataset_delete = dataset.select([i for i in dataset.columns if i != 'label'])

In [75]:
dataset_delete.show(5)

+---+---+---+---+---+-----------------+
|_c0|_c1|_c2|_c3|_c4|         features|
+---+---+---+---+---+-----------------+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|
+---+---+---+---+---+-----------------+
only showing top 5 rows



In [60]:
dataset._c0

Column<b'_c0'>


In [61]:
new_dataset = dataset.withColumn('_c00', dataset['_c0'] + 2)

In [62]:
new_dataset

DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double, _c00: double]

In [63]:
new_dataset.show()

+---+---+---+---+---+-----------------+-----+----+
|_c0|_c1|_c2|_c3|_c4|         features|label|_c00|
+---+---+---+---+---+-----------------+-----+----+
|5.1|3.5|1.4|0.2|1.0|[5.1,3.5,1.4,0.2]|  0.0| 7.1|
|4.9|3.0|1.4|0.2|1.0|[4.9,3.0,1.4,0.2]|  0.0| 6.9|
|4.7|3.2|1.3|0.2|1.0|[4.7,3.2,1.3,0.2]|  0.0| 6.7|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0| 6.6|
|5.0|3.6|1.4|0.2|1.0|[5.0,3.6,1.4,0.2]|  0.0| 7.0|
|5.4|3.9|1.7|0.4|1.0|[5.4,3.9,1.7,0.4]|  0.0| 7.4|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0| 6.6|
|5.0|3.4|1.5|0.2|1.0|[5.0,3.4,1.5,0.2]|  0.0| 7.0|
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0| 6.4|
|4.9|3.1|1.5|0.1|1.0|[4.9,3.1,1.5,0.1]|  0.0| 6.9|
|5.4|3.7|1.5|0.2|1.0|[5.4,3.7,1.5,0.2]|  0.0| 7.4|
|4.8|3.4|1.6|0.2|1.0|[4.8,3.4,1.6,0.2]|  0.0| 6.8|
|4.8|3.0|1.4|0.1|1.0|[4.8,3.0,1.4,0.1]|  0.0| 6.8|
|4.3|3.0|1.1|0.1|1.0|[4.3,3.0,1.1,0.1]|  0.0| 6.3|
|5.8|4.0|1.2|0.2|1.0|[5.8,4.0,1.2,0.2]|  0.0| 7.8|
|5.7|4.4|1.5|0.4|1.0|[5.7,4.4,1.5,0.4]|  0.0| 7.7|
|5.4|3.9|1.3|0.4|1.0|[5.4,3.9,1

In [31]:
type(dataset.select('_c0'))

pyspark.sql.dataframe.DataFrame

In [33]:
dataset_cl= dataset.select('_c0').collect()

In [34]:
dataset_cl

[Row(_c0=5.1),
 Row(_c0=4.9),
 Row(_c0=4.7),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=5.4),
 Row(_c0=4.6),
 Row(_c0=5.0),
 Row(_c0=4.4),
 Row(_c0=4.9),
 Row(_c0=5.4),
 Row(_c0=4.8),
 Row(_c0=4.8),
 Row(_c0=4.3),
 Row(_c0=5.8),
 Row(_c0=5.7),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=5.7),
 Row(_c0=5.1),
 Row(_c0=5.4),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.0),
 Row(_c0=5.0),
 Row(_c0=5.2),
 Row(_c0=5.2),
 Row(_c0=4.7),
 Row(_c0=4.8),
 Row(_c0=5.4),
 Row(_c0=5.2),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=5.0),
 Row(_c0=5.5),
 Row(_c0=4.9),
 Row(_c0=4.4),
 Row(_c0=5.1),
 Row(_c0=5.0),
 Row(_c0=4.5),
 Row(_c0=4.4),
 Row(_c0=5.0),
 Row(_c0=5.1),
 Row(_c0=4.8),
 Row(_c0=5.1),
 Row(_c0=4.6),
 Row(_c0=5.3),
 Row(_c0=5.0),
 Row(_c0=7.0),
 Row(_c0=6.4),
 Row(_c0=6.9),
 Row(_c0=5.5),
 Row(_c0=6.5),
 Row(_c0=5.7),
 Row(_c0=6.3),
 Row(_c0=4.9),
 Row(_c0=6.6),
 Row(_c0=5.2),
 Row(_c0=5.0),
 Row(_c0=5.9),
 Row(_c0=6.0),
 Row(_c0=6.1),
 Row(_c0=5.6),
 Row(_c0=6.7),
 Row(_c0=5

In [39]:
# dataset_list = [i[0] for i in dataset_cl]  # OK
dataset_list = [i['_c0'] for i in dataset_cl]  # OK

In [40]:
dataset_list

[5.1,
 4.9,
 4.7,
 4.6,
 5.0,
 5.4,
 4.6,
 5.0,
 4.4,
 4.9,
 5.4,
 4.8,
 4.8,
 4.3,
 5.8,
 5.7,
 5.4,
 5.1,
 5.7,
 5.1,
 5.4,
 5.1,
 4.6,
 5.1,
 4.8,
 5.0,
 5.0,
 5.2,
 5.2,
 4.7,
 4.8,
 5.4,
 5.2,
 5.5,
 4.9,
 5.0,
 5.5,
 4.9,
 4.4,
 5.1,
 5.0,
 4.5,
 4.4,
 5.0,
 5.1,
 4.8,
 5.1,
 4.6,
 5.3,
 5.0,
 7.0,
 6.4,
 6.9,
 5.5,
 6.5,
 5.7,
 6.3,
 4.9,
 6.6,
 5.2,
 5.0,
 5.9,
 6.0,
 6.1,
 5.6,
 6.7,
 5.6,
 5.8,
 6.2,
 5.6,
 5.9,
 6.1,
 6.3,
 6.1,
 6.4,
 6.6,
 6.8,
 6.7,
 6.0,
 5.7,
 5.5,
 5.5,
 5.8,
 6.0,
 5.4,
 6.0,
 6.7,
 6.3,
 5.6,
 5.5,
 5.5,
 6.1,
 5.8,
 5.0,
 5.6,
 5.7,
 5.7,
 6.2,
 5.1,
 5.7,
 6.3,
 5.8,
 7.1,
 6.3,
 6.5,
 7.6,
 4.9,
 7.3,
 6.7,
 7.2,
 6.5,
 6.4,
 6.8,
 5.7,
 5.8,
 6.4,
 6.5,
 7.7,
 7.7,
 6.0,
 6.9,
 5.6,
 7.7,
 6.3,
 6.7,
 7.2,
 6.2,
 6.1,
 6.4,
 7.2,
 7.4,
 7.9,
 6.4,
 6.3,
 6.1,
 7.7,
 6.3,
 6.4,
 6.0,
 6.9,
 6.7,
 6.9,
 5.8,
 6.8,
 6.7,
 6.7,
 6.3,
 6.5,
 6.2,
 5.9]

In [41]:
type(dataset_list[0])

float

In [49]:
# 删除某些无用列
new_dataset = dataset.drop('_c0', '_c1', '_c2', '_c3', '_c4')

In [50]:
new_dataset.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
+-----------------+-----+
only showing top 5 rows



In [51]:
# 新_模型训练
lr_2 = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel_2 = lr_2.fit(new_dataset)

In [52]:
prediction_2 = lrModel_2.transform(new_dataset)

In [53]:
prediction_2.show(150)

+-----------------+-----+--------------------+--------------------+----------+
|         features|label|       rawPrediction|         probability|prediction|
+-----------------+-----+--------------------+--------------------+----------+
|[5.1,3.5,1.4,0.2]|  0.0|[1.50553010141029...|[0.80168217364268...|       0.0|
|[4.9,3.0,1.4,0.2]|  0.0|[1.20141808462763...|[0.69508498886403...|       0.0|
|[4.7,3.2,1.3,0.2]|  0.0|[1.43534662127354...|[0.76732997443658...|       0.0|
|[4.6,3.1,1.5,0.2]|  0.0|[1.34214957417624...|[0.73710671332729...|       0.0|
|[5.0,3.6,1.4,0.2]|  0.0|[1.60975088260149...|[0.82670286305514...|       0.0|
|[5.4,3.9,1.7,0.4]|  0.0|[1.52181555631473...|[0.82246378795528...|       0.0|
|[4.6,3.4,1.4,0.3]|  0.0|[1.53372993913371...|[0.80131748475294...|       0.0|
|[5.0,3.4,1.5,0.2]|  0.0|[1.43782002857652...|[0.78118697494117...|       0.0|
|[4.4,2.9,1.4,0.2]|  0.0|[1.28319035129927...|[0.70504407252594...|       0.0|
|[4.9,3.1,1.5,0.1]|  0.0|[1.30272547919305...|[0.727

In [54]:
trainingSummary = lrModel_2.summary
trainingSummary

In [55]:
trainingSummary.falsePositiveRateByLabel

[0.0, 0.11, 0.05]

In [56]:
trainingSummary.accuracy

0.8933333333333333

In [65]:
(trainingData, testData) = dataset.randomSplit([0.8, 0.2], 123)
# print("Training Dataset Count: " + str(trainingData.count()))
# print("Test Dataset Count: " + str(testData.count()))
# trainingData.show(10)
testData.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|
|4.4|3.0|1.3|0.2|1.0|[4.4,3.0,1.3,0.2]|  0.0|
|4.4|3.2|1.3|0.2|1.0|[4.4,3.2,1.3,0.2]|  0.0|
|4.6|3.6|1.0|0.2|1.0|[4.6,3.6,1.0,0.2]|  0.0|
|4.7|3.2|1.6|0.2|1.0|[4.7,3.2,1.6,0.2]|  0.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [66]:
trainingData.show(5)

+---+---+---+---+---+-----------------+-----+
|_c0|_c1|_c2|_c3|_c4|         features|label|
+---+---+---+---+---+-----------------+-----+
|4.3|3.0|1.1|0.1|1.0|[4.3,3.0,1.1,0.1]|  0.0|
|4.5|2.3|1.3|0.3|1.0|[4.5,2.3,1.3,0.3]|  0.0|
|4.6|3.1|1.5|0.2|1.0|[4.6,3.1,1.5,0.2]|  0.0|
|4.6|3.2|1.4|0.2|1.0|[4.6,3.2,1.4,0.2]|  0.0|
|4.6|3.4|1.4|0.3|1.0|[4.6,3.4,1.4,0.3]|  0.0|
+---+---+---+---+---+-----------------+-----+
only showing top 5 rows



In [67]:
# 模型训练
lr = LogisticRegression(maxIter=20, regParam=0.3, elasticNetParam=0)
lrModel = lr.fit(trainingData)

In [76]:
# 模型预测
predictions = lrModel.transform(testData)
# print(prediction)
predictions.show(10)

# ROC score
# evaluator = BinaryClassificationEvaluator()
# evaluator.evaluate(prediction)

+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|_c0|_c1|_c2|_c3|_c4|         features|label|       rawPrediction|         probability|prediction|
+---+---+---+---+---+-----------------+-----+--------------------+--------------------+----------+
|4.4|2.9|1.4|0.2|1.0|[4.4,2.9,1.4,0.2]|  0.0|[0.97587696525420...|[0.66435501880071...|       0.0|
|4.4|3.0|1.3|0.2|1.0|[4.4,3.0,1.3,0.2]|  0.0|[1.07468573243426...|[0.69738164615681...|       0.0|
|4.4|3.2|1.3|0.2|1.0|[4.4,3.2,1.3,0.2]|  0.0|[1.22655912321016...|[0.74546466374773...|       0.0|
|4.6|3.6|1.0|0.2|1.0|[4.6,3.6,1.0,0.2]|  0.0|[1.55117580808555...|[0.82793089816996...|       0.0|
|4.7|3.2|1.6|0.2|1.0|[4.7,3.2,1.6,0.2]|  0.0|[1.08632343975477...|[0.70280652702730...|       0.0|
|4.9|2.4|3.3|1.0|2.0|[4.9,2.4,3.3,1.0]|  2.0|[-0.3580763884417...|[0.22319947512485...|       2.0|
|5.0|3.0|1.6|0.2|1.0|[5.0,3.0,1.6,0.2]|  0.0|[0.86283058089978...|[0.62295572416110...|       0.0|
|5.0|3.4|1

In [77]:
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [78]:
accuracy

0.8857142857142857

In [72]:
preSummary = prediction.summary
preSummary

<bound method DataFrame.summary of DataFrame[_c0: double, _c1: double, _c2: double, _c3: double, _c4: string, features: vector, label: double, rawPrediction: vector, probability: vector, prediction: double]>

In [73]:
preSummary.accuracy

AttributeError: 'function' object has no attribute 'accuracy'

In [74]:
preSummary.weightedRecall

AttributeError: 'function' object has no attribute 'weightedRecall'